<a href="https://colab.research.google.com/github/adityasahu88/DL_PROJECTS/blob/main/CNN_CT_VS_DOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

image5_b9b4c3cb25.avif

**Pain Points:**<br/>
- The Simple cat and Dog classification we are unable to process 25k images
- We use Image data Generator  to process the entire images.
- We are processing the entire image i.e all the background of cat and dog images. Model should focus on Dog or cat from a image
- It do not do any feature selection as Deep Learning uses the feature selection
- For Simple Cat and Dog images We have added 4 Hidden layers and 34Millions Param are used to classification.


- The solution for the above two pain points is **CNN**

**CNN - Convolutional Neural Network**

- **Convolutional Layer:** It applies a set of filters (also called kernels) to the input image or previous layer output. These filters slide across the image (or feature map) and detect features such as edges, textures, or patterns.
- Each filter generates a feature map that highlights particular features in the image.
-We can say it helps us to reduce the size of the image by using filter on the original images and extract the important feature from the images.
-Filters are the small matrix and the size of a matrix is 3*3 generally but it is a hyper-parameter. The values in the filter matrix is random initially and values updated by backward and forward propagation as it is happending in case of SGD the weight and bias are random initally .
- Filter matrix applies  a dot product operation with the image pixels and  generate features and highlights the important features.
- Filter matrix or kernel is always smaller then the original matrix.


- CNN will do the feature engineering by applying filter matrix and max pooling filter over the input images


image9_fbc98b6c6e.avif

**Pooling layer**<br/>

- The goal of the pooling layer is to pull the most significant features from the convoluted matrix. This is done by applying some aggregation operations, which reduce the dimension of the feature map (convoluted matrix), hence reducing the memory used while training the network. Pooling is also relevant for mitigating overfitting.

The most common aggregation functions that can be applied are:

- Max pooling, which is the maximum value of the feature map
- Sum pooling corresponds to the sum of all the values of the feature map
- Average pooling is the average of all the values.

image4_332794a93a.avif

**Padding :**<br/>
- Suppose input image is small and the strides or steps is equal to 2 then adding extra pixels to the input image.
- Padding is the process of adding extra pixels (usually zeros) around the input image before applying the convolution operation.
- Padding adds extra rows and columns to the input image (or feature map) before applying the convolution filter.
- It is typically added symmetrically on all sides of the image.
The number of rows and columns added depends on the padding size.

Also, the dimension of the feature map becomes smaller as the pooling function is applied.

The last pooling layer flattens its feature map so that it can be processed by the fully connected layer.

**Example**
 - Suppose if the image original size is 5 * 5 and filter matrix size is 3 * 3 and step size is 1 then it will return a features 3*3

- **Formula**- (I-F+S+P)/S
- (5-3+1+0)/1 =3

- the steps size is generally 1 but again it is a hyperparameter. if the step size 1 there is maximum overlapping

**Example**<br/>
- Suppose We have a image of 220 * 220 * 3 means 220 height, 220 width and 3 channels
- We will apply CNN over the image then it will reduce the image to 7 * 7 * 512
- This means  the height and width of the image becomes 7 * 7 respectively and 512 is the dimensionality.
-  the 3 color chanels (RGB) has been reduced to 1 channel and the 512 will the different features selection of the images as in some image we will focus on edges or in some image we will focus on the contrast. The single image will be converted into multiple images.
- The final objective is to reduce the height and width of the image by applying filter matrix and the max pooling filter over the images and generate multiple flavours of same image say 512 may be think like image augmentation.
-After that we will connect it to the flatten layers and then fully connected neural network

In [ ]:
# lets implement the concept with the help of python

In [2]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle

In [3]:
!kaggle datasets download bhavikjikadara/dog-and-cat-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/dog-and-cat-classification-dataset
License(s): apache-2.0
 99% 771M/775M [00:06<00:00, 114MB/s]
100% 775M/775M [00:06<00:00, 116MB/s]


In [4]:
!unzip -q dog-and-cat-classification-dataset.zip

In [5]:
import pandas as pd
import numpy as np
import cv2
import matplotlib
import tensorflow as tf


In [6]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                      validation_split=0.10
                                                      )

In [7]:
batch_size =64

In [8]:
train_idg = idg.flow_from_directory("PetImages",
                                    subset="training",batch_size=64,
                                    interpolation="bilinear",
                                    target_size=(150,150))

Found 22500 images belonging to 2 classes.


In [9]:
val_idg= idg.flow_from_directory("PetImages",
                                    subset="validation",batch_size=64,
                                    interpolation="bilinear",
                                    target_size=(150,150))

Found 2498 images belonging to 2 classes.


In [10]:
# Create the model

model =tf.keras.models.Sequential()

model.add(tf.keras.layers.Input(shape =(150,150,3),name ="Input_layer"))

# Apply the filters/Kernel over the input images
# In keras the filters are called CONV2D


model.add(tf.keras.layers.Conv2D(filters = 16, # Apply 16 filter over the image nd generate 16 features
                                 kernel_size =(3,3),
                                 strides=(1,1),
                                 padding="valid",
                                 activation= tf.keras.activations.relu,
                                 name ="Conv1"))

# (150,150,3),(3 *3 )  ----> (150-3+1)/1 = 148  --> (148,148,16)

# Apply the pooling over the input image

model.add(tf.keras.layers.MaxPool2D(pool_size =(2,2),
                                    strides=(2,2),
                                    padding="valid",
                                    name ="Pool1"))

# (148,148,16) -->(74,74,16)

model.add(tf.keras.layers.Conv2D(filters = 32, # Apply 16 filter over the image nd generate 16 features
                                 kernel_size =(3,3),
                                 strides=(1,1),
                                 padding="valid",
                                 activation= tf.keras.activations.relu,
                                 name ="Conv2"))

# (74,74,16),(3 *3 )  ----> (74-3+1)/1 = 72  --> (72,72,32)

# Apply the pooling over the input image

model.add(tf.keras.layers.MaxPool2D(pool_size =(2,2),
                                    strides=(2,2),
                                    padding="valid",
                                    name="Pool2"))

#  (72,72,32) -->36  --> (36,36,32)

model.add(tf.keras.layers.Conv2D(filters = 32, # Apply 16 filter over the image nd generate 16 features
                                 kernel_size =(3,3),
                                 strides=(1,1),
                                 padding="valid",
                                 activation= tf.keras.activations.relu,
                                 name ="Conv3"))

# (36,36,32),(3 *3 )  ----> (36-3+1)/1 = 34  --> (34,43,32)

# Apply the pooling over the input image

model.add(tf.keras.layers.MaxPool2D(pool_size =(2,2),
                                    strides=(2,2),
                                    padding="valid",
                                    name="Pool3"))

#  (34,34,32) -->17 --> (17,17,32)





# Add the Flatten layers
model.add(tf.keras.layers.Flatten(name ="Flatten"))



# Earlier  the input param for flatten layer is 150*150*3 now it reduces to 36*36*32 after applying CNN

# Add the Hidden layer


model.add(tf.keras.layers.Dense(64,activation=tf.keras.activations.relu,name='HL1'))
model.add(tf.keras.layers.Dense(32,activation=tf.keras.activations.relu,name='HL2'))
model.add(tf.keras.layers.Dense(8,activation=tf.keras.activations.relu,name='HL3'))

# Add the Output layer

model.add(tf.keras.layers.Dense(2,activation=tf.keras.activations.softmax,name='Output_Layer'))

model.summary()




Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Conv1 (Conv2D)                       │ (None, 148, 148, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool1 (MaxPooling2D)                 │ (None, 74, 74, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2 (Conv2D)                       │ (None, 72, 72, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool2 (MaxPooling2D)                 │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv3 (Conv2D)                       │ (None, 34, 34, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Pool3 (MaxPooling2D)                 │ (None, 17, 17, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flatten (Flatten)                    │ (None, 9248)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ HL1 (Dense)                          │ (None, 64)                  │         591,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ HL2 (Dense)                          │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ HL3 (Dense)                          │ (None, 8)                   │             264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output_Layer (Dense)                 │ (None, 2)                   │              18 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 608,634 (2.32 MB)

 Trainable params: 608,634 (2.32 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics =['accuracy'])

In [12]:
model.fit(train_idg,epochs=10,validation_data=val_idg,batch_size=batch_size)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
115/352 ━━━━━━━━━━━━━━━━━━━━ 41s 176ms/step - accuracy: 0.5271 - loss: 0.6916

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


352/352 ━━━━━━━━━━━━━━━━━━━━ 82s 218ms/step - accuracy: 0.5393 - loss: 0.6893 - val_accuracy: 0.5845 - val_loss: 0.6766
Epoch 2/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 69s 196ms/step - accuracy: 0.5944 - loss: 0.6692 - val_accuracy: 0.6141 - val_loss: 0.6624
Epoch 3/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 69s 195ms/step - accuracy: 0.6137 - loss: 0.6566 - val_accuracy: 0.6037 - val_loss: 0.6618
Epoch 4/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 69s 197ms/step - accuracy: 0.6204 - loss: 0.6474 - val_accuracy: 0.6441 - val_loss: 0.6441
Epoch 5/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 81s 194ms/step - accuracy: 0.6359 - loss: 0.6385 - val_accuracy: 0.6449 - val_loss: 0.6328
Epoch 6/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 69s 196ms/step - accuracy: 0.6481 - loss: 0.6253 - val_accuracy: 0.6621 - val_loss: 0.6162
Epoch 7/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 69s 197ms/step - accuracy: 0.6657 - loss: 0.6141 - val_accuracy: 0.6657 - val_loss: 0.6258
Epoch 8/10
352/352 ━━━━━━━━━━━━━━━━━━━━ 82s 198ms/step - accuracy: 0.6876 - loss: 0.5919 - val